In [1]:
# #import .env variables
# from dotenv import dotenv_values
# config = dotenv_values(".env")
# config["USERNAME"]

In [2]:
#imports
import boto3
import requests
import pandas as pd 
from tqdm import tqdm

In [3]:
# def authenticate_user():
#     PENNSIEVE_URL = "https://api.pennsieve.io"
#     email = config["USERNAME"]
#     password = config["PASSWORD"]

#     r = requests.get(f"{PENNSIEVE_URL}/authentication/cognito-config")
#     r.raise_for_status()
#     cognito_app_client_id = r.json()["userPool"]["appClientId"]
#     cognito_region = r.json()["userPool"]["region"]

#     cognito_client = boto3.client(
#     "cognito-idp",
#     region_name=cognito_region,
#     aws_access_key_id="",
#     aws_secret_access_key="")
    
#     login_response = cognito_client.initiate_auth(
#     AuthFlow="USER_PASSWORD_AUTH",
#     AuthParameters={"USERNAME": email, "PASSWORD": password},
#     ClientId=cognito_app_client_id)
    
#     api_key = login_response["AuthenticationResult"]["AccessToken"]
    
#     r = requests.get(f"{PENNSIEVE_URL}/user", headers={"Authorization": f"Bearer {api_key}"})
#     r.raise_for_status()
#     print(r.json())
#     return api_key 
    
# api_key = authenticate_user()

In [4]:
def get_published_datasets():
    url = "https://api.pennsieve.io/discover/search/records"
    querystring = {"model": "award"}
    headers = {"Accept": "application/json"}
    response = requests.request("GET", url, headers=headers, params=querystring)
    response.raise_for_status()
    response = response.json()
    querystring = {"limit": response["totalCount"], "model": "award"}
    response = requests.request("GET", url, headers=headers, params=querystring)
    response = response.json()
    list_of_datasets = response["records"].copy()
    return list_of_datasets

list_of_datasets = get_published_datasets()
list_of_datasets[0]

for item in tqdm(list_of_datasets):
    url = f"https://api.pennsieve.io/discover/datasets/{item['datasetId']}"
    headers = {"Accept": "application/json"}
    response = requests.request("GET", url, headers=headers)
    response.raise_for_status()
    response = response.json()
    item['version'] = response['version']
    item['versionPublishedAt'] = response['versionPublishedAt']
    item['datasetDOI'] = 'https://dx.doi.org/' + response['doi']
    item['tags'] = response['tags']
    item['contributors'] = response['contributors']

100%|██████████████████████████████████████████████████████████████████████████████████| 98/98 [00:35<00:00,  2.77it/s]


In [5]:
#change loop variant on prod
# for item in list_of_datasets[:1]:
#     url = f"https://api.pennsieve.io/discover/datasets/{item['datasetId']}/versions/{item['version']}/files"
#     querystring = {"path":"files/dataset_description.xlsx"}
#     headers = {"Accept": "application/json"}
#     response = requests.request("GET", url, headers=headers, params=querystring)
#     response = response.json()
#     item["datasetDescriptionPackageId"] = response["sourcePackageId"]
#     print(response)

In [6]:
#change loop variant on prod
# loop_variant = list_of_datasets[:2]
loop_variant = list_of_datasets[:5]

for item in tqdm(loop_variant):
    url = "https://api.pennsieve.io/zipit/discover"    
    payload = {"data": {
        "paths": ["files/dataset_description.xlsx"],
        "version": item['version'],
        "datasetId": item['datasetId']
    }}
    response = requests.request("POST", url, json=payload)
    response.raise_for_status()
    local_filename = "dataset_description.xlsx"
    totalbits = 0
    if response.status_code == 200:
        with open(local_filename, 'wb') as f:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:
                    totalbits += 1024
#                     print("Downloaded",totalbits*1025,"KB...")
                    f.write(chunk)
        try: 
            xl = pd.ExcelFile("dataset_description.xlsx")
            df = xl.parse("Sheet1")

            # arrays to hold dois for each dataset
            originating_article_array = []
            protocol_array = []

            # get all dois from dataframe
            for index, row in df.iterrows():
                row_val = row['Metadata element']
                if (row_val.find('Originating Article') != -1):
                    count = 0
                    for col_val in row:
                        if (count > 2):
                            try: 
                                if col_val.find('doi.org') != -1:
                                    pos = col_val.find('http')
                                    if pos != -1:
                                        col_val = col_val[pos:]
                                    originating_article_array.append(col_val)
                            except:
                                pass
                        count = count + 1
                if (row_val.find('Protocol') != -1):
                    count = 0
                    for col_val in row:
                        if (count > 2):
                            try: 
                                if (col_val.find('doi.org') != -1):
                                    pos = col_val.find('http')
                                    if pos != -1:
                                        col_val = col_val[pos:]
                                    protocol_array.append(col_val)
                            except:
                                pass
                        count = count + 1
            item["originatingArticleDOI"] = originating_article_array
            item["protocolsDOI"] = protocol_array
        except: 
            print(item)
            item["originatingArticleDOI"] = []
            item["protocolsDOI"] = []

for item in loop_variant:
    print( item)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:04<00:00,  1.09it/s]

{'datasetId': 35, 'version': 3, 'model': 'award', 'properties': {'description': 'PROJECT ABSTRACTThe peripheral and central elements of the respiratory control system are not “fixed,” but undergo sustained(neuroplastic) circuit reorganization to optimize function. This system can selectively utilize unique afferentmodalities and brainstem neural pathways to elicit episodic, coordinated airway protective behaviors (e.g.cough, laryngeal adduction). Neuroplasticity is induced and undermined by inflammation, transient afferentfeedback, or CNS injury. As a result, breathing responses and airway protective behaviors are altered in waysthat can be adaptive or maladaptive. Existing models of the brainstem network and sensory control systemregulating breathing and airway protection do not explain changes in responses caused by neuroplasticity insensory, central integrating and efferent motor elements of the control system. This knowledge gap concerningperipheral and central circuit-based proces

In [35]:
for item in loop_variant:
    print(item["datasetId"], item["version"], item["originatingArticleDOI"],  item["protocolsDOI"])

64 4 [] ['dx.doi.org/10.17504/protocols.io.6bvhan6']
121 1 [] ['dx.doi.org/10.17504/protocols.io.bafnibme']
24 2 ['https://doi.org/10.1111/nmo.13380'] []
31 3 [] []
54 1 [] ['https://dx.doi.org/10.17504/protocols.io.wqmfdu6']
65 6 [] ['dx.doi.org/10.17504/protocols.io.bh4dj8s6']
27 2 [] []
35 3 ['https://doi.org/10.3389/fphys.2018.00785'] []
58 1 [] []
55 1 [] []
127 1 [] []
137 1 [] []
103 4 [] []
63 2 ['https://doi.org/10.1371/journal.pcbi.1002061'] []
23 2 [] ['dx.doi.org/10.17504/protocols.io.5nkg5cw']
43 5 [] ['https://dx.doi.org/10.17504/protocols.io.9gbh3sn', 'https://dx.doi.org/10.17504/protocols.io.y3tfynn', 'https://dx.doi.org/10.17504/protocols.io.bjfzkjp6']
29 6 [] []
119 1 [] []
56 1 [] ['https://dx.doi.org/10.17504/protocols.io.wjrfcm6']
76 3 [] []
10 3 [] []
59 1 ['https://doi.org/10.1007/s00441-018-2957-0'] []
97 1 ['https://doi.org/10.1088/1741-2552/ab7ad4'] []
123 1 [] ['https://doi.org/10.17504/protocols.io.2irgcd6']
130 1 ['https://doi.org/10.1038/s41598-021-81822-3

In [ ]:
import boto3
import requests
import pandas as pd 
from tqdm import tqdm

In [ ]:
def get_list_of_datasets_with_metadata(list_of_datasets):
    
    # get list of datasets with awards associated with it
    url = "https://api.pennsieve.io/discover/search/records"
    querystring = {"model": "award"}
    headers = {"Accept": "application/json"}

    #test request to find out how many total datsets are present
    response = requests.request(
        "GET", url, headers=headers, params=querystring)
    response.raise_for_status()
    response = response.json()

    # get all
    querystring = {"limit": response["totalCount"], "model": "award"}
    response = requests.request(
        "GET", url, headers=headers, params=querystring)
    response.raise_for_status()
    response = response.json()
    list_of_datasets = response["records"].copy()

    url = ""
    querystring = ""
    headers = ""
    response = ""

    # get latest version of each dataset
    for item in tqdm(list_of_datasets):
        url = f"https://api.pennsieve.io/discover/datasets/{item['datasetId']}"
        headers = {"Accept": "application/json"}
        response = requests.request("GET", url, headers=headers)
        response.raise_for_status()
        response = response.json()
        item['name'] = response['name']
        item['description'] = response['description']
        item['version'] = response['version']
        item['versionPublishedAt'] = response['versionPublishedAt']
        item['datasetDOI'] = 'https://dx.doi.org/' + response['doi']
        item['tags'] = response['tags']
        item['contributors'] = response['contributors']

    # extract metadata information
    for item in tqdm(list_of_datasets):
        # get the actual dataset_description.xlsx file.
        # seems to be the best way for pandas to read
        url = "https://api.pennsieve.io/zipit/discover"
        payload = {"data": {
            "paths": ["files/dataset_description.xlsx"],
            "version": item['version'],
            "datasetId": item['datasetId']
        }}
        response = requests.request("POST", url, json=payload)
        response.raise_for_status()
        local_filename = "dataset_description.xlsx"
        totalbits = 0

        # write binary data to a file that is readable in pandas
        if response.status_code == 200:
            with open(local_filename, 'wb') as f:
                for chunk in response.iter_content(chunk_size=1024):
                    if chunk:
                        totalbits += 1024
                        # print("Downloaded",totalbits*1025,"KB...")
                        f.write(chunk)

            # only reading through xlsx files at the moment.
            # could be extended for json and csv (rare instances by SPARC curation standards) if time permits
            try:
                xl = pd.ExcelFile("dataset_description.xlsx")
                df = xl.parse("Sheet1")

                # arrays to hold dois for each dataset
                originating_article_array = []
                protocol_array = []

                # get all dois from dataframe
                for index, row in df.iterrows():
                    row_val = row['Metadata element']
                    if (row_val.find('Originating Article') != -1):
                        count = 0
                        for col_val in row:
                            if (count > 2):
                                try:
                                    if col_val.find('doi.org') != -1:
                                        pos = col_val.find('http')
                                        if pos != -1:
                                            col_val = col_val[pos:]
                                        originating_article_array.append(
                                            col_val)
                                except:
                                    pass
                            count = count + 1
                    if (row_val.find('Protocol') != -1):
                        count = 0
                        for col_val in row:
                            if (count > 2):
                                try:
                                    if (col_val.find('doi.org') != -1):
                                        pos = col_val.find('http')
                                        if pos != -1:
                                            col_val = col_val[pos:]
                                        protocol_array.append(col_val)
                                except:
                                    pass
                            count = count + 1
                item["originatingArticleDOI"] = originating_article_array
                item["protocolsDOI"] = protocol_array
            except:
                # for any xlsx files that are corrupted or cannot be read, ignore
                item["originatingArticleDOI"] = []
                item["protocolsDOI"] = []

    return list_of_datasets

datset_list = []
datset_list = get_list_of_datasets_with_metadata(list_of_datasets)
print(datset_list)